<h1> UAS</h1>

In [69]:
# Seluruh package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.svm import SVC

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix

kode soal nomor -1

In [70]:
dataset = pd.read_excel('dataset/list data no.1.xlsx').iloc[:,:9]
dataset.head() # Tampilkan 5 dataset teratas

,CardID,TrxDate,Amount
0,1000152,2021-05-19,932841
1,1000158,2021-03-25,881117
2,1000156,2021-04-25,368989
3,1000115,2021-06-14,621767
4,1000164,2021-01-02,451551


In [71]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [72]:
print(dataset['TrxDate'].min(), dataset['TrxDate'].max())

2021-01-01 00:00:00 2021-06-30 00:00:00


In [73]:
sd = dt.datetime(2021,12,10)
dataset['hist']=sd - dataset['TrxDate']
dataset['hist'].astype('timedelta64[D]')
dataset['hist']=dataset['hist'] / np.timedelta64(1, 'D')
dataset.head(10)

,CardID,TrxDate,Amount,hist
0,1000152,2021-05-19,932841,205.0
1,1000158,2021-03-25,881117,260.0
2,1000156,2021-04-25,368989,229.0
3,1000115,2021-06-14,621767,179.0
4,1000164,2021-01-02,451551,342.0
5,1000136,2021-03-14,103939,271.0
6,1000147,2021-03-20,380979,265.0
7,1000118,2021-06-19,688837,174.0
8,1000167,2021-04-17,833738,237.0
9,1000160,2021-03-26,82160,259.0


In [74]:
dataset=dataset[dataset['hist'] < 730]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.8 KB


In [75]:
rfmTable = dataset.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(10)

,recency,frequency,monetary_value
CardID,,,
1000101,250.0,1,203955
1000102,181.0,3,2120942
1000103,193.0,1,696250
1000104,205.0,2,612707
1000105,163.0,5,2590460
1000106,199.0,2,825166
1000107,282.0,3,1603483
1000108,209.0,3,2393851
1000109,329.0,2,677799


In [76]:
quartiles = rfmTable.quantile(q=[0.15,0.30,0.45])
print(quartiles, type(quartiles))

      recency  frequency  monetary_value
0.15    176.5        1.0        584261.0
0.30    191.0        2.0        825166.0
0.45    201.0        2.0       1058823.5 <class 'pandas.core.frame.DataFrame'>


In [77]:
def RClass(x,p,d):
    if x <= d[p][0.15]:
        return 4
    elif x <= d[p][0.30]:
        return 3
    elif x <= d[p][0.45]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.15]:
        return 1
    elif x <= d[p][0.30]:
        return 2
    elif x <= d[p][0.45]: 
        return 3
    else:
        return 4

In [78]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [79]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,250.0,1,203955,1,1,1,111
1000102,181.0,3,2120942,3,4,4,344
1000103,193.0,1,696250,2,1,2,212
1000104,205.0,2,612707,1,2,2,122
1000105,163.0,5,2590460,4,4,4,444
1000106,199.0,2,825166,2,2,2,222
1000107,282.0,3,1603483,1,4,4,144
1000108,209.0,3,2393851,1,4,4,144
1000109,329.0,2,677799,1,2,2,122


In [80]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,250.0,1,203955,1,1,1,111
1000102,181.0,3,2120942,3,4,4,344
1000103,193.0,1,696250,2,1,2,212
1000104,205.0,2,612707,1,2,2,122
1000105,163.0,5,2590460,4,4,4,444
1000106,199.0,2,825166,2,2,2,222
1000107,282.0,3,1603483,1,4,4,144
1000108,209.0,3,2393851,1,4,4,144
1000109,329.0,2,677799,1,2,2,122


In [81]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Langganan: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Pembeli banyak: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Beresiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Lost: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Pelanggan berpotensial: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Butuh Perhatian: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan Tetap:  8
Langganan:  36
Pembeli banyak:  39
Beresiko:  39
Lost:  5
Pelanggan berpotensial:  8
Butuh Perhatian:  6


Kode soal nomor - 2

In [82]:
# Baca dataset yang ditentukan
dataset = pd.read_excel('dataset/list data no.2.xlsx').iloc[:,:9]
dataset.head() # Tampilkan 5 dataset teratas

,Customer ID,Umur,Income,Product_holding,Saving,Deposit,KK,Tab Bisnis,Limit Kredit Mortgage
0,1234561175,27,18044724,3,30496813,22079895,856297,0,4537131469
1,1234561810,53,41844607,3,5459323,0,599572,17479146,2515048708
2,1234561778,52,34437700,2,34261341,0,2743516,0,2021012050
3,1234561915,40,45774220,4,27771816,18639840,4534694,18489681,617757979
4,1234561363,55,35452808,2,38657479,0,0,11176695,4157711017


In [83]:
# Pisahkan antara x dan y (target)
x = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [84]:
# Test size 30% dan otomatis training selisih nya ialah 70%
training_data, testing_data, training_label, testing_label = train_test_split(x, y,test_size=0.3)

In [85]:
# Standarized dataset (opsional)
training_data = StandardScaler().fit_transform(training_data)
testing_data = StandardScaler().fit_transform(testing_data)

In [86]:
# Lakukan PCA
pca = PCA(.95)
training_data = pca.fit_transform(training_data)
testing_data = pca.fit_transform(testing_data)
pca.explained_variance_ratio_.cumsum()

array([0.23840857, 0.38772867, 0.5155979 , 0.64135634, 0.76040368,
       0.873036  , 0.98423799])

In [87]:
# Lakukan Multiple Linear Regression
multiple_liner_reg = linear_model.LinearRegression()
multiple_liner_reg.fit(training_data, training_label)

LinearRegression()

In [88]:
label_prediction = multiple_liner_reg.predict(testing_data)

In [89]:
# Prediksi kepada 10 buah data
multiple_liner_reg.predict(testing_data[0:10])

array([2.74908996e+09, 3.05396368e+09, 3.11193267e+09, 3.13083372e+09,
       3.23830972e+09, 3.23781298e+09, 2.97952686e+09, 3.18360819e+09,
       2.98729295e+09, 2.82756146e+09])

In [90]:
# - Akurasi Section - 
# Skor terbaik yang ialah adalah 1,0 dan 
# bisa negatif karena modelnya bisa lebih buruk saat akan di fit. 
# Sebuah model konstan yang selalu memprediksi nilai yang diharapkan dari label, mengabaikan fitur input, akan mendapatkan skor R2 0,0.
score_model=r2_score(testing_label, label_prediction)
print("R2 Score : ",score_model)
print("Mean Squared Error == ",mean_squared_error(testing_label, label_prediction))
print("Root Mean Squared Error ==",np.sqrt(mean_squared_error(testing_label, label_prediction)))

R2 Score :  -0.024291662276724413
Mean Squared Error ==  2.613324590471701e+18
Root Mean Squared Error == 1616578049.6071637


In [91]:
# Hasil yang dikembangkan cukup buruk jika mengabaikan atau hanya mengikuti default yang dilakukan oleh function
# Maka diperlukan tunning dan pemilihan alias selection fitur yang sesuai yang mempengaruhi secara baik input dengan label
# R square dapat bernilai negatif ketika model yang dipilih tidak mengikuti tren data, 
# sehingga menghasilkan kecocokan yang lebih buruk daripada garis horizontal. 
# Hal ini biasanya terjadi ketika ada kendala baik pada intersep atau kemiringan garis regresi linier.

Kode soal nomor - 3

In [92]:
# Baca dataset yang ditentukan
dataset = pd.read_excel('dataset/list data no.3.xlsx')
dataset.head() # Tampilkan 5 dataset teratas

,Customer ID,Age,Income,Education,Occupation,Income.1,Average Balance,Appraisal,Industri,Pelunasan Kredit
0,1234561427,2,4,4,5,5,5,1,1,1
1,1234561518,1,5,2,5,4,3,4,3,0
2,1234561282,1,5,5,4,5,3,3,4,1
3,1234561064,1,2,3,1,4,2,2,3,0
4,1234561043,5,4,2,3,4,3,1,4,1


In [93]:
# Pisahkan antara x dan y (target)
x = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [94]:
# Test size 30% dan otomatis training selisih nya ialah 70%
training_data, testing_data, training_label, testing_label = train_test_split(x, y,test_size=0.3)

In [95]:
# Standarized dataset/ (opsional)
training_data = StandardScaler().fit_transform(training_data)
testing_data = StandardScaler().fit_transform(testing_data)

In [96]:
# Penggunaan PCA
# pca = PCA(n_components=2)
training_data = pca.fit_transform(training_data)
testing_data = pca.fit_transform(testing_data)

In [97]:
svm = SVC(kernel='linear')
svm.fit(training_data, training_label)
label_prediction = svm.predict(testing_data)

In [98]:
# - Akurasi Section-
print(classification_report(testing_label, label_prediction))

# Untuk output sangat jelek jika dilihat dari hasil precission dan recall yang didapatkan
# Hasil akurasi tanpa tunning sekitar 45%-50% karena data dari features kurang mempengaruhi label
# Untuk meningkatkan akurasi dapat dilakukan dengan seleksi feature yaitu membuang features yang tidak berguna
# Jika menggunakan PCA hanya meningkatkan akurasi sekitar 45% menjadi 51-55%

              precision    recall  f1-score   support

           0       0.47      0.35      0.40       141
           1       0.53      0.65      0.58       159

    accuracy                           0.51       300
   macro avg       0.50      0.50      0.49       300
weighted avg       0.50      0.51      0.50       300

